In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute
from qiskit.circuit import Parameter
import numpy as np
import qiskit
from qiskit.quantum_info import Pauli
from qiskit import opflow
from qiskit.opflow import PauliSumOp

### 2-site system
### (1/2) p12 x1 x2 + (1/2) p21 x2 x1

### (1/2) p12 (1 + Z1) / 2 (1 + Z2) / 2 + (1/2) p12 (1 + Z1) / 2 (1 + Z2) / 2
### p12 / 4 + Z1 (1 / 4 p12) + Z2 (1 / 4 p12)


# \sum_ij Wij xi xj
# xi/xj = 0, 1
# xi -> (1 + Zi) / 2, xj -> (1 + Zj) / 2
# pi xi^2 -> pi [(1 + Zi) / 2]^2 = pi (1 + Zi) / 2 = pi/2 + pi Zi / 2
# (1/2) \sum_ij pij xi xj ->pij x (1 + Zi) / 2 x (1 + Zj) / 2 = pij/4 + pij Zi/4 + pij Zj/4 + pijZi Zj / 4

# on diagonal of W, stay resulting coefficients in front of Z_i
# on off-diagonal of W, stay resu. coefficients in front of Z_i Z_j, but also divided by 2

def to_matrix(onsite, pair, n_qubits):
    W = np.zeros((n_qubits, n_qubits), dtype=np.float64)
    for onsite_term in onsite:
        i, pi = onsite_term
        W[i, i] += pi / 2.
        
    for pair_term in pair:
        i, j, pij = pair_term
        W[i, j] += pij / 8.
        W[j, i] += pij / 8
            
        W[i, i] += pij / 8
        W[j, j] += pij / 8
        
    return W


def transform_interaction_to_qiskit_format(n_qubits, hamiltonian):
    r"""Generate Hamiltonian for the problem
    """
    
    onsite = hamiltonian.onsite
    pair = hamiltonian.pair
    
    def get_shift(onsite, pair):
        shift = 0.
        for onsite_term in onsite:
            _, pi = onsite_term
            
            shift += pi / 2.
            
        for pair_term in pair:
            _, _, pij = pair_term
            
            shift += pij / 4.
        return shift

    shift = get_shift(onsite, pair)
    
    W = to_matrix(onsite, pair, n_qubits)
    
    pauli_list = []

    for i in range(n_qubits):
        for j in range(n_qubits):
            if np.isclose(W[i, j], 0.0):
                continue
            x_p = np.zeros(n_qubits, dtype=bool)
            z_p = np.zeros(n_qubits, dtype=bool)
            z_p[i] = True
            z_p[j] = True
            pauli_list.append([W[i, j], Pauli((z_p, x_p))])

    pauli_list = [(pauli[1].to_label(), pauli[0]) for pauli in pauli_list]
    return PauliSumOp.from_list(pauli_list), shift

def evaluate_cost(solution, hamiltonian):
    energy = 0
    for single_term in hamiltonian.onsite:
        energy += single_term[1] * (solution[single_term[0]] == 1)
        
    for pair_term in hamiltonian.pair:
        energy += pair_term[2] * (solution[pair_term[0]] == 1) * (solution[pair_term[1]] == 1)
        
    return energy


def index_to_spin(index, n_qubits):
    return (((np.array([index]).reshape(-1, 1) & (1 << np.arange(n_qubits)))) > 0).astype(np.int64)

def bruteforce_solution(n_qubits, hamiltonian):
    energies = []
    bit_representations = []
    for idx in range(2 ** n_qubits):
        solution = index_to_spin(idx, n_qubits)[0]
        
        
        bit_representations.append(solution.copy())
        energies.append(evaluate_cost(solution, hamiltonian))
        
    energies = np.array(energies)
    bit_representations = np.array(bit_representations)
    
    return np.sort(energies), bit_representations[np.argsort(energies)]



def maxcut_obj(x, G):
    """
    Given a bitstring as a solution, this function returns
    the number of edges shared between the two partitions
    of the graph.
    
    Args:
        x: str
           solution bitstring
           
        G: networkx graph
        
    Returns:
        obj: float
             Objective
    """
    obj = 0
    for i, j in G.edges():
        if x[i] != x[j]:
            obj -= 1
            
    return obj


def compute_expectation(counts, ham):
    """
    Computes expectation value based on measurement results
    
    Args:
        counts: dict
                key as bitstring, val as count
        ham: hamiltonian instance
    Returns:
        avg: float
             expectation value
    """
    
    def cast_to_int_array(bitstring):
        return np.asarray([int(y) for y in (list(bitstring))])
    
    avg = 0
    sum_count = 0
    for bitstring, count in counts.items():
        obj = evaluate_cost(cast_to_int_array(bitstring), ham) ## BEWARE: order of bitstring
        avg += obj * count
        sum_count += count
    #print('CURRENT LOSS:', avg / sum_count)
    return avg/sum_count


# We will also bring the different circuit components that
# build the qaoa circuit under a single function
def create_qaoa_circ(ham, theta):
    #print('CURRENT PARAMETERS DURING THE OPTIMIZATION ARE:', theta)
    """
    Creates a parametrized qaoa circuit
    
    Args:  
        ham: networkx graph
        theta: list
               unitary parameters
                     
    Returns:
        qc: qiskit circuit
    """
    
    nqubits = ham.n_qubits
    p = len(theta) // 2  # number of alternating unitaries
    qc = QuantumCircuit(nqubits)

    beta = theta[:p]
    gamma = theta[p:]  # TODO add rz parameters
    
    # initial_state
    for i in range(nqubits):
        qc.h(i)
    
    for irep in range(p):
        # problem unitary
        for pair in list(ham.pair):
            qc.rzz(2 * gamma[irep] * pair[2], pair[0], pair[1])
            
        for onsite in list(ham.onsite):
            qc.rz(2 * gamma[irep] * onsite[1], onsite[0])

        # mixer unitary
        for i in range(nqubits):
            qc.rx(2 * beta[irep], i)
            
    qc.measure_all()
        
    return qc

# Finally we write a function that executes the circuit on the chosen backend
def get_expectation(ham, p, shots=512):
    
    """
    Runs parametrized circuit
    
    Args:
        hham: hamiltonian
        p: int,
           Number of repetitions of unitaries
    """
    
    backend = Aer.get_backend('qasm_simulator')
    backend.shots = shots
    
    def execute_circ(theta):
        qc = create_qaoa_circ(ham, theta)
        counts = backend.run(qc, seed_simulator=10, 
                             nshots=512).result().get_counts()
        
        return compute_expectation(counts, ham)
    
    return execute_circ

In [2]:
from collections import OrderedDict
from qiskit import Aer
from qiskit import algorithms
from qiskit.algorithms import QAOA
from qiskit.opflow import StateFn
from qiskit.algorithms.optimizers import ADAM, COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms import VQE
from qiskit.circuit.library import TwoLocal


def most_frequent_strings(state_vector, num_most_frequent):
    """Compute the most likely binary string from state vector.
    Args:
        state_vector (numpy.ndarray or dict): state vector or counts.
    Returns:
        numpy.ndarray: binary string as numpy.ndarray of ints.
    """
    most_frequent_strings = [x[0] for x in sorted(state_vector.items(), \
                                                  key=lambda kv: kv[1])[-num_most_frequent:]]
    return [np.asarray([int(y) for y in (list(binary_string))]) for binary_string in most_frequent_strings]




class hamiltonian(object):
    def __init__(self, onsite, pair, n_qubits):
        self.onsite = onsite
        self.pair = pair
        self.n_qubits = n_qubits
        return

n_qubits = 5

def get_random_Hamiltonian(n_qubits):
    onsite = []
    pair = []
    
    for i in range(n_qubits):
        onsite.append((i, np.random.uniform(-2, 2)))
        
    for i in range(n_qubits):
        for j in range(i + 1, n_qubits):
            pair.append((i, j, np.random.uniform(-2, 2)))
    return hamiltonian(onsite, pair, n_qubits)

ham = get_random_Hamiltonian(n_qubits)
energies, bits = bruteforce_solution(n_qubits, ham)

print('ALL BRUTE FORCE SOLUTIONS')

for en, xi in zip(energies, bits):
    print('BF string:', xi, 'cost:', en)


p = n_qubits
from scipy.optimize import minimize
expectation = get_expectation(ham, p=p)

res = minimize(expectation, np.ones(2 * p), method='COBYLA')
print('FINAL PARAMETERS:', res.x)
backend = Aer.get_backend('aer_simulator')
backend.shots = 512

qc_res = create_qaoa_circ(ham, res.x)

counts = backend.run(qc_res, seed_simulator=10).result().get_counts()
print(counts)
x = most_frequent_strings(counts, 4)
#x = [1 - xi[::-1] for xi in x]


print('\n\n\nTESTING THE QUANTUM OUTPUT')
for xi in x:
    print('QC string:', xi, 'cost:', evaluate_cost(xi, ham))





#qubit_op, offset = transform_interaction_to_qiskit_format(n_qubits, ham)


#optimizer = COBYLA()
#
#vqe = QAOA(optimizer, quantum_instance=Aer.get_backend('qasm_simulator'))# 
#ansatz = TwoLocal(qubit_op.num_qubits, 'ry', 'cz', reps=5, entanglement='full')
#vqe = VQE(ansatz, optimizer, quantum_instance=Aer.get_backend('qasm_simulator'))

#result = vqe.compute_minimum_eigenvalue(qubit_op)



ALL BRUTE FORCE SOLUTIONS
BF string: [1 1 1 1 1] cost: -11.700225681447979
BF string: [0 1 1 1 1] cost: -8.976658036780147
BF string: [1 1 1 1 0] cost: -8.927399140993463
BF string: [1 0 1 1 1] cost: -8.22169327192045
BF string: [0 1 1 1 0] cost: -8.038991716611635
BF string: [1 1 0 1 1] cost: -7.086161680615619
BF string: [1 1 1 0 1] cost: -5.6972287810233055
BF string: [1 0 1 1 0] cost: -5.691291775118881
BF string: [1 1 0 1 0] cost: -5.276000247784563
BF string: [0 1 0 1 0] cost: -5.217890244035383
BF string: [0 1 0 1 1] cost: -5.192891456580433
BF string: [0 0 1 1 1] cost: -4.8780033304253685
BF string: [0 0 1 1 0] cost: -4.182762053909803
BF string: [1 0 1 0 1] cost: -4.032654481778096
BF string: [0 1 1 0 1] cost: -3.9657333105634507
BF string: [0 1 1 0 0] cost: -3.9440297871843266
BF string: [1 1 1 0 0] cost: -3.8403650373581772
BF string: [1 0 0 1 1] cost: -3.65613090301227
BF string: [1 0 1 0 0] cost: -2.418215781765913
BF string: [1 0 0 1 0] cost: -2.0883945138341597
BF string

In [42]:
0 1  1 0 | 0 110 -> Mz -> 1001

1.0

In [93]:
result.eigenstate

{'00001': 0.03125,
 '00010': 0.04419417382415922,
 '00111': 0.03125,
 '01001': 0.03125,
 '01100': 0.06987712429686843,
 '01110': 0.05412658773652741,
 '10000': 0.04419417382415922,
 '10010': 0.04419417382415922,
 '10100': 0.23593232610221093,
 '10101': 0.08838834764831845,
 '10110': 0.07654655446197431,
 '10111': 0.03125,
 '11000': 0.10364452469860624,
 '11100': 0.8766725086940961,
 '11101': 0.34938562148434216,
 '11110': 0.10825317547305482}

In [103]:
dir(vqe)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_ansatz',
 '_ansatz_params',
 '_callback',
 '_check_operator_ansatz',
 '_circuit_sampler',
 '_cost_fn',
 '_eval_aux_ops',
 '_eval_count',
 '_eval_time',
 '_expectation',
 '_get_eigenstate',
 '_gradient',
 '_include_custom',
 '_initial_point',
 '_max_evals_grouped',
 '_optimizer',
 '_parameterized_circuits',
 '_quantum_instance',
 '_ret',
 'ansatz',
 'callback',
 'cleanup_parameterized_circuits',
 'compute_minimum_eigenvalue',
 'construct_circuit',
 'construct_expectation',
 'expectation',
 'find_minimum',
 'get_energy_evaluation',
 'get_optimal_circuit',
 'get_optimal_c